# Diving into Pinecone

This notebook uses the latest versions of the Pinecone library.

In [ ]:
# pip install -q pinecone-client

In [ ]:
# pip install --upgrade -q pinecone-client

In [ ]:
# pip show pinecone-client

In [1]:
# authenticating to Pinecone. 
# the API KEY is in .env
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from pinecone import Pinecone

# Initilizing and authenticating the pinecone client
pc = Pinecone()
# pc = Pinecone(api_key='YOUR_API_KEY')

# checking authentication
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'churchill-speech-0e53e0f.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'churchill-speech',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

## Working with Pinecone Indexes

In [3]:
# listing all indexes
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'churchill-speech-0e53e0f.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'churchill-speech',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [22]:
index_name = 'langchain'
# getting a complete description of a specific index:
pc.describe_index(index_name)

{'dimension': 1536,
 'host': 'langchain-0e53e0f.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [23]:
# getting a list with the index names 
pc.list_indexes().names()

['langchain']

In [24]:
# deleting an index
pc.delete_index(index_name)
if index_name in pc.list_indexes().names():
    print(f'Deleting index {index_name} ... ')
    pc.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist!')

Index langchain does not exist!


In [25]:
# creating a Pinecone index 
# only one index permitted on the free plan

from pinecone import PodSpec
index_name = 'langchain'

if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=PodSpec(
            environment='gcp-starter'
        )
    )
    print('Index created! 😊')
else:
    print(f'Index {index_name} already exists!')

Creating index langchain
Index created! 😊


In [27]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Working with Vectors

In [28]:
# inserting vectors
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_name = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [29]:
# updating vectors
index.upsert(vectors=[('c', [0.5] * 1536)])

{'upserted_count': 1}

In [30]:
# fetching vectors
# index = pc.Index(index_name)
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'c': {'id': 'c',
                   'values': [0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
             

In [31]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [39]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [40]:
# querying a non-existing vector returns an empty vector
index.fetch(ids=['x']) 

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [37]:
# querying vectors
query_vector = [random.random() for _ in range(1536)]

In [38]:
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'd', 'score': 0.75184989, 'values': []},
             {'id': 'e', 'score': 0.749291062, 'values': []},
             {'id': 'a', 'score': 0.741943121, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

## Namespaces

In [41]:
# index.describe_index_stats()
index = pc.Index('langchain')

import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [42]:
# partition the index into namespaces
# creating a new namespace
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [43]:
vectors = [[random.random() for _ in range(1536)] for v in range(2)]
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [52]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0001,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 10}

In [49]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [47]:
index.fetch(ids=['x'], namespace='first-namespace')


{'namespace': 'first-namespace',
 'usage': {'read_units': 1},
 'vectors': {'x': {'id': 'x',
                   'values': [0.545609295,
                              0.0231331177,
                              0.00833628327,
                              0.985678613,
                              0.169645756,
                              0.714971185,
                              0.828501165,
                              0.214708775,
                              0.424497873,
                              0.882438421,
                              0.682920516,
                              0.901985049,
                              0.168136388,
                              0.324502051,
                              0.160465628,
                              0.96300745,
                              0.333950162,
                              0.407352597,
                              0.990926862,
                              0.718174458,
                              0.860408247,
   

In [53]:
index.delete(ids=['x'], namespace='first-namespace')

{}

In [54]:
index.delete(delete_all=True, namespace='first-namespace')

{}

In [55]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 7e-05,
 'namespaces': {'': {'vector_count': 5},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 7}